In [26]:
import numpy as np
import pandas as pd
import scipy.sparse
import matplotlib.pyplot as plt
%matplotlib inline

# Custom metric is implemented here
from scorer import scorer
from tqdm import tqdm
from joblib import Parallel, delayed
from sklearn.linear_model import LogisticRegression

# Load data

# Use custom dtypes for efficiency
dtypes = {'id1': np.int16, 'id2': np.int16, 'id3': np.int16, 'user_id': np.int32, 'date': np.int16}

train = pd.read_csv('train.csv.zip', dtype=dtypes)
# train_const = train.copy()
train.shape

(46996828, 5)

* ~~Посмотреть что он открывал 4 недели назад и больше но не открывал за 3 недели и есть ли тут пересечения~~
* Предсказывать юзеров которые каждую неделю открывают минимум 5 новых категорий

In [ ]:
# drop 'inactive' and 'overactive' users 

# user_count = np.unique(train_const.user_id[train_const.date>(np.max(train_const.date)-7)], return_counts=True)
user_count = np.unique(train.user_id[train.date>(np.max(train.date)-7)], return_counts=True)
user_count = pd.DataFrame({
    'user_id':user_count[0], 'counts':user_count[1]
})

#print(user_count.shape)
treshold_count_low = 8
treshold_count_high = 50
user_count = user_count[(user_count.counts>=treshold_count_low) & (user_count.counts<treshold_count_high)]


# user_count = user_count[:10000]
# user_count = user_count[0:5500]
print(user_count.shape)

train = pd.merge(train, user_count, how='left', on ='user_id')
train = train.dropna()
# print(train.shape, len(pd.unique(train.user_id)))

del user_count

Забавный факт - увеличивая скор на валидации с помощью изменения трешхолда treshold_count_low и второго, на сабмите скор падает
### Пример:
    
    валидация - паблик
    2600 - 1900
    3600 - 1600

In [2]:
import telepyth # push notif in telegram

%telepyth -t 1260389131217015787

Use token 1260389131217015787.


In [3]:
date_validation_start = train.date.max() - 6

def calculate_target(data, date_test_start):
    '''
        This function returns a dictionary of type {user: items_list}
        Such that user viewed an item in testing period, 
        but did not view it within the last 3 weeks of train period.
    '''
    
    test_mask = (data.date >= date_test_start) & (data.date < date_test_start + 7)
    last_3weeks_mask = (data.date >= date_test_start - 21 + 1) & (data.date < date_test_start)
    
    # Items that used viewed during test period
    items_test = data[test_mask].groupby('user_id').id3.apply(set)
    
    # Items, that user viewd in last 3 weeks
    user_last_3weeks = data[last_3weeks_mask].groupby('user_id').id3.apply(set)
    
    # Get table, where for each `user_id` we have both items from test period and 3 weeks
    joined = items_test.reset_index().merge(user_last_3weeks.reset_index(), on=['user_id'], how='left')
    joined.set_index('user_id', inplace=True)
    
    # Remove the items, which the user viewed during last 3 weeks 
    target = {}
    for user_id, (id3_x, id3_y) in joined.iterrows():   
        items = id3_x if id3_y is np.nan else id3_x - id3_y
        if items != set(): target.update({user_id: items})

    return target

y_val_dict = calculate_target(train, date_validation_start)
# np.save('y_val_dict', y_val_dict)
# y_val_dict = np.load('y_val_dict.npy').item()

y_val_dict1 = calculate_target(train, date_validation_start-7)
y_val_dict2 = calculate_target(train, date_validation_start-7*2)


ind_list = []
for k, v in y_val_dict.items():
    try:
        if (len(v) > 4) and (len(y_val_dict1[k]) > 4) and (len(y_val_dict2[k]) > 4): 
            ind_list.append(k)
    except:
        continue
len(ind_list)

np.save("ind_list", ind_list)

In [ ]:
num = 53979

mask_train = (train.date < date_validation_start - 7) & (train.date > date_validation_start - 7*4)
mask_test = (train.date < date_validation_start) & (train.date >= date_validation_start - 7*3)

# users = train.loc[mask_test].user_id.value_counts().index[:num]
# mask_users = train.user_id.isin(users)

# mask_train = mask_train & mask_users
# mask_test = mask_test & mask_users
# users_mask = train.user_id < 10000
# mask_train = mask_train & users_mask


def get_feats(data):
    '''
        Builds sparse matrix using users' history.
    '''
    return scipy.sparse.coo_matrix(([1] * data.shape[0], (data.user_id, data.id3)), 
                                    shape =[data.user_id.max()+1, data.id3.max()+1]).tocsr()

def get_target_matrix(X, target_dict):
    '''
        Builds sparse matrix using dictionary.
    '''
    indptr = [0]
    indices = []
    data = []
    vocabulary = {}

    ks = []
    for k in tqdm(range(X.user_id.max()+1)):
        d = target_dict.get(k, [])
        for y in d:
            indices.append(y)
            data.append(1)
        indptr.append(len(indices))
    return scipy.sparse.csr_matrix((data, indices, indptr), dtype=int, shape =[X.user_id.max()+1, X.id3.max()+1])

X_train = get_feats(train.loc[mask_train])
X_test = get_feats(train.loc[mask_test])

y_train_dict = calculate_target(train, date_validation_start - 7)
y_train = get_target_matrix(train.loc[mask_train], y_train_dict)
y_test = get_target_matrix(train.loc[mask_test], y_val_dict)

In [ ]:

dot = np.dot(X_test_nmf, H)
# тут логика такая: раскладываем матрицу "оценок" на фичи юзеров и фичи айди3, потом раскладываем такую же матрицу на новых 
# данных и умножаем на фичи айдишек

users = train.loc[mask_test].user_id.value_counts().index[:num]
# users = pd.Int64Index(idx)
ans_inds =  np.argsort(dot[users])
# test_inds_dict =  {k: list(ans_inds[i, -5:]) for i,k in enumerate(users)}
# scorer(y_val_dict, test_inds_dict, num_users=num/0.05)

last_3weeks = train.loc[mask_test].loc[train.loc[mask_test].date >= train.loc[mask_test].date.max() - 21 + 1]
y_not = last_3weeks.groupby('user_id').id3.apply(set)

y_pred = {}
num_users=num/0.05
for u_idx, user in tqdm(enumerate(users)):
    items_not = y_not.get(user, [])
    items_pred = []
    i = 1
    while len(items_pred) < 5:
        if not ans_inds[u_idx, -i] in items_not:
            items_pred += [ans_inds[u_idx, -i]]
    
        i += 1
    y_pred.update({user: items_pred})

num_users = 53979/0.05
score = scorer(y_val_dict, y_pred, num_users)
%telepyth 'Very magic, wow!\n' + str(score)

In [ ]:
mask_sub = train.date >= date_validation_start
X_sub = get_feats(train.loc[mask_sub])

from sklearn.decomposition import NMF
nmf = NMF(n_components=5, init='random', random_state=0)
x_train_nmf = nmf.fit_transform(X_test)
y_test_nmf = nmf.components_
x_sub_nmf = nmf.transform(X_sub)
dot = np.dot(x_sub_nmf, y_test_nmf)

users = train.loc[mask_test].user_id.value_counts().index[:num]
# users = pd.Int64Index(idx)
ans_inds =  np.argsort(dot[users])
# test_inds_dict =  {k: list(ans_inds[i, -5:]) for i,k in enumerate(users)}
# scorer(y_val_dict, test_inds_dict, num_users=num/0.05)

last_3weeks = train.loc[mask_test].loc[train.loc[mask_test].date >= train.loc[mask_test].date.max() - 21 + 1]
y_not = last_3weeks.groupby('user_id').id3.apply(set)

y_pred = {}
for u_idx, user in tqdm(enumerate(users)):
    items_not = y_not.get(user, [])
    items_pred = []
    i = 1
    while len(items_pred) < 5:
        if not ans_inds[u_idx, -i] in items_not:
            items_pred += [ans_inds[u_idx, -i]]
    
        i += 1
    y_pred.update({user: items_pred})
# num_users = 49130/0.05
# score = scorer(y_val_dict, y_pred, num_users)
# %telepyth 'Very magic, wow!\n' #+ str(score)
len(y_pred)

In [ ]:
y_pred_df = pd.DataFrame.from_records(y_pred).T.reset_index()
y_pred_df.columns = ['user_id', 'id3_1', 'id3_2', 'id3_3', 'id3_4', 'id3_5']

y_pred_df.to_csv('y_pred.csv', index=False)

Это попытка предсказания каких юзеров выбирать, я думаю на этом стоит остановиться, может быть кор метрикой

In [ ]:
# # del Y_test, Y_train
# Y_train = np.ndarray((1179572))
# for key in tqdm(y_train_dict):
#     Y_train[key] = len(y_train_dict[key])

# Y_test = np.ndarray((1179572))
# for key in tqdm(y_val_dict):
#     Y_test[key] = len(y_val_dict[key])

# Y_train = Y_train.astype('int32')
# Y_test = Y_test.astype('int32')

# import lightgbm as lgb

# verbose = 1
# plot = 1
# params = {
# #     "max_bin": 1024,
#     "learning_rate": 0.01,
#     "boosting_type": "goss",
#     "objective": "regression",
#     'num_iterations':10000,
# #     "metric": "auc",
#     "num_leaves": 10000,
#     "verbose": 1,
# #     "min_data": 100,
# #     "boost_from_average": True
# }

# d_train = lgb.Dataset(X_train_nmf, Y_train)
# # d_valid = lgb.Dataset(X_test, label=y_test)
# model = lgb.train(params, d_train)
# # model.save_model('regressin')
# # model = lgb.Booster(model_file='regressin')

# y_pred = model.predict(X_test_nmf)
# from sklearn.metrics import mean_squared_error
# # print(roc_auc_score(y_test.values, y_pred))
# print(mean_squared_error(y_pred ,Y_test)**0.5)

# df_fi = pd.DataFrame(model.feature_name(), columns=['feature'])
# df_fi['importance'] = list(model.feature_importance('gain'))
# df_fi.sort_values('importance', ascending=False, inplace=True)
# # print(df_fi)
# if plot:
#     plt.figure()
#     df_fi.head(10).plot(kind='barh', x='feature', y='importance')
#     plt.title('LightGBM Feature Importance')
#     plt.xlabel('relative importance')
#     plt.show()

# idx = (y_pred).argsort()[:53979]
# idx # ids to test main task
# %telepyth 'lgb!\n' + str(mean_squared_error(y_pred ,Y_test)**0.5)